### Ports

https://wiki.openmod-initiative.org/wiki/SciGRID_gas

maybe
http://users.dimi.uniud.it/~massimo.franceschet/ns/syllabus/make/gas/gas.html

https://github.com/mihai-craita/countries_center_box

https://github.com/EnergieID/entsoe-py/blob/master/entsoe/mappings.py

country codes and examples:
https://thesmartinsights.com/how-to-query-data-from-the-entso-e-transparency-platform-using-python/

```python
# vessel = !python scripts/get_vessels.py
data = !python scripts/get_vessels.py data.csv
df = pd.DataFrame(data=data)[0].str.split(',',expand=True)
df.head(3)
```

https://stackoverflow.com/questions/68762104/plotly-adding-scatter-geo-points-and-traces-on-top-of-density-mapbox

```python
from nltk.corpus import wordnet as wn
from itertools import product

wordx, wordy = "cat","dog"
sem1, sem2 = wn.synsets(wordx), wn.synsets(wordy)

maxscore = 0
for i,j in list(product(*[sem1,sem2])):
    score = i.wup_similarity(j) # Wu-Palmer Similarity
    maxscore = score if maxscore < score else maxscore
```

```python
import sys
!{sys.executable} -m pip install geopy
```

```python
from geopy.distance import geodesic
origin = (30.172705, 31.526725)  # (latitude, longitude) don't confuse
dist = (30.288281, 31.732326)
print(geodesic(origin, dist).kilometers)  # 23.576805481751613
```

In [ ]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize
from bs4 import BeautifulSoup
from shapely.geometry import Polygon, LineString
import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
from pathlib import Path
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path

# geo libraries
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt
from geopy.distance import geodesic
from geopandas import GeoDataFrame
import shapely.ops
import shapely.geometry
from shapely.geometry import Polygon, LineString
cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#vessel = pd.read_csv('./data/lng_tankers_lat.csv', index_col=False)
conn = sqlite3.connect('file.db')
vessel = pd.read_sql_query("SELECT * FROM lng", conn)

vessel.sort_values(by='DWT', ascending=False, inplace=True)

geometry = [Point(xy) for xy in zip(vessel.lon, vessel.lat)]
# vessel = vessel.drop(['lon', 'lat'], axis=1)
vessel = GeoDataFrame(vessel, crs="EPSG:4326", geometry=geometry)
vessel.head()

vessel = vessel[~(vessel['date'] == '2022-12-20')]
vessel.head()

In [ ]:
def get_port_distance(lat, lon, max_distance):
    """
    latitude and longitude of the port and maxiaml distance of the vessel to the port
    """
    dist_list = []
    for index, row in vessel.iterrows():
        try:
            origin = (row['lat'], row['lon'])  # (latitude, longitude) don't confuse
            dist = (lat, lon)
            distance = round(geodesic(origin, dist).kilometers)
            # print(row['imo'], distance)
            if distance < max_distance:
                dist_list.append(row['imo'])
        except Exception as e:
            # print(e)
            pass
    return dist_list

def radius_around_port(lat, lon, radius_in_meter):
    radius = gpd.GeoDataFrame({'ship': f'test', 'geometry': [Point(lon, lat)]}, crs='EPSG:4326')
    radius = radius.to_crs(epsg=7855).buffer(radius_in_meter).to_crs(epsg=4326)
    boundary_geojson = gpd.GeoSeries(radius)
    region = boundary_geojson.geometry.unary_union
    return region

def get_line(poly):
    try:
        #ls = shapely.geometry.LineString(df) # shapely.geometry.LineString(shapely.ops.unary_union(df).exterior.coords)
        b = poly.boundary.coords
        ls = shapely.geometry.LineString(b)
        #linestrings = [LineString(b[k:k+2]) for k in range(len(b) - 1)]
        #print([list(ls.coords) for ls in linestrings])
        lats, lons = ls.coords.xy
        return pd.Series([lats, lons])
    except Exception as e:
        # print(e)
        return pd.Series([np.nan, np.nan])


p = pd.read_csv('./data/GIE_LNGs_V1.csv', index_col=False)
p['bubble'] = p.apply(lambda x: radius_around_port(x['lat'], x['lon'], radius_in_meter=80000), axis=1)
p[['lats','lons']] = p.apply(lambda x: get_line(x['bubble']), axis=1)
ports = p.copy()
ports['vessels_close'] = ports.apply(lambda x: get_port_distance(x['lat'], x['lon'], 30), axis=1)
ports.tail(5)

### References Konkurrenz
https://lng.team/#prices
https://medium.com/analytics-vidhya/how-to-create-a-simple-web-app-for-a-vessels-eta-api-using-python-db485182a8c2


In [ ]:
vessel.tail()

In [ ]:
# %load scripts/draw_map.py

def add_ship(fig, df, col="#76b5c5", size=12):
    try:
        print(df.head())
        #template=f"<br>Name: {df['imo']}</b><extra></extra>"
        fig.add_scattermapbox(
            lat=df['lat'].tolist(), lon=df['lon'].tolist(), mode='markers',
            #hovertemplate=template,
            marker_size=size, marker_color=col,
            line=dict(width=6),opacity= 0.4, hoverinfo='none',
            showlegend=False,)
    
    except Exception as e:
        print(e)

        
def add_radius(fig, df):
    try:
        radius_in_meter = 10000
        fig.add_trace(go.Scattermapbox(
                mode="lines",
                lon=list(df['lats']),
                lat=list(df['lons']),
                name=f"{radius_in_meter} m radius",
                hoverinfo='skip',
                showlegend=False,
                marker={'size': 15, 'color': 'red', 'opacity': 0.6}))
    except Exception as e:
        pass

    
def add_ports(fig, df, col="#bbb5c5", symbol="harbor", size=12):
    try:
        if symbol == "harbor":
            fig.add_scattermapbox(
            name='Home', lat=df['lat'].tolist(), lon=df['lon'].tolist(),
            mode='markers+text',
            marker=dict(size=size, color=col),
            text=df['Name'].tolist(),
            textposition="bottom center",
            hoverinfo='text',
            textfont=dict(color='black', size=10),
            showlegend=False,)
                
        else:
            #template=f"<br>Name: {df['imo']}</b><extra></extra>"
            fig.add_scattermapbox(
                name='Home',
                lat=df['lat'].tolist(), lon=df['lon'].tolist(),
                mode='markers',
                #hovertemplate=template,
                #marker_size=size, 
                #marker_color=col,
                marker=dict(size=size, symbol=symbol, color=col),
                #line=dict(width=6),
                #if symbol == "harbor":
                #    text=df['Name'].tolist(),
               #    textposition="bottom center",
                #    hoverinfo='text',
               #     textfont=dict(color='black', size=10),

                showlegend=False,)

    except Exception as e:
        print(e)

def world_map(vessel, ports):
    fig = go.Figure()
    
    # add traces
    add_ports(fig, ports, col="#76b5c5", symbol="harbor", size=16)
    add_ports(fig, vessel, col="#be4d25", symbol="marker", size=10)
    
    for index, row in ports.iterrows():
        add_radius(fig, row)

        
    fig.update_layout(mapbox_style="light", mapbox_accesstoken=
                      "pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                      mapbox_zoom=3, mapbox_center={"lat": 47.809490, "lon": 13.055010})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(title_text = 'Global LNG ports and LNG carrier fleets')
    fig.show()


world_map(vessel, ports)

In [ ]:
dft = vessel.copy()
dft= dft.groupby('imo').agg({'lat':lambda x: list(x), 'lon':lambda x: list(x)})
dft.tail()

def add_trace(fig, df, col="#76b5c5", size=12):
    try:
       # template=f"<br>Name: {df['imo']}</b><extra></extra>"
        fig.add_scattermapbox(
            lat=df['lat'], lon=df['lon'], mode="markers+text+lines",
            #hovertemplate=template,
        showlegend=False
        )
    
    except Exception as e:
        print(e)

def add_trace2(fig, df, col="#76b5c5", size=12):
    try:
       # template=f"<br>Name: {df['imo']}</b><extra></extra>"
        fig.add_scattermapbox(
            lat=df['lat'], lon=df['lon'], mode="markers+text+lines",
            #hovertemplate=template,
        showlegend=False
        )
    
    except Exception as e:
        print(e)

def add_ports(fig, df, col="#bbb5c5", symbol="harbor", size=12):
    try:
        if symbol == "harbor":
            fig.add_scattermapbox(
            name='Home', lat=df['lat'].tolist(), lon=df['lon'].tolist(),
            mode='markers+text',
            marker=dict(size=size, color=col),
            text=df['Name'].tolist(),
            textposition="bottom center",
            hoverinfo='text',
            textfont=dict(color='black', size=10),
            showlegend=False,)
        else:
            #template=f"<br>Name: {df['imo']}</b><extra></extra>"
            fig.add_scattermapbox(
                name='Home',
                lat=df['lat'].tolist(), lon=df['lon'].tolist(),
                mode='markers',
                marker=dict(size=size, symbol=symbol, color=col),
                showlegend=False,)
    except Exception as e:
        print(e)


def world_map_V2(vessel, ports):
    fig = go.Figure()
    
    # add traces
    add_ports(fig, ports, col="#76b5c5", symbol="harbor", size=16)
    

    #add_trace2(fig, dft)
    
    for index, row in dft.iterrows():
        add_trace(fig, row)

        
    fig.update_layout(mapbox_style="light", mapbox_accesstoken=
                      "pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                      mapbox_zoom=3, mapbox_center={"lat": 47.809490, "lon": 13.055010})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 40, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.update_layout(title_text = 'Global LNG ports and LNG carrier fleets')
    fig.show()


world_map_V2(vessel, ports)

In [ ]:
fig = px.line_mapbox(vessel, lat="lat", lon="lon", color="imo", zoom=3, height=300)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(mapbox_style="light", mapbox_accesstoken=
                  "pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg",
                  mapbox_zoom=3, mapbox_center={"lat": 47.809490, "lon": 13.055010})
fig.show()

In [ ]:
dft2 = vessel.copy()
vessel.head()
dft2.dropna(subset=['lon', 'lat'], inplace=True)
dft2.head(600)

dft3= dft2.groupby('imo').agg({'lat':lambda x: list(x), 'lon':lambda x: list(x), 'geometry':lambda x: list(x), 'date':lambda x: list(x)})

def generate_linestring(geometry):
    #LineString([Point(0, 0), Point(1, 1)]).wkt
    try:
        # works also with comprehensive list
        return LineString(geometry).wkt
    except Exception as e:
        return -1

dft3['line'] = dft3.apply(lambda x: generate_linestring(x['geometry']), axis=1)

dft3.reset_index(inplace=True)
dft3 = dft3.rename(columns = {'index':'imo'})
dft3.tail()

lats = []
lons = []
names = []
for index, row in dft3.iterrows():
    for i in row['lat']:
        lats = np.append(lats, i)
        
    for i in row['lon']:
        lons = np.append(lons, i)
        
    for i in range(len(row['lat'])):
        names = np.append(names, row['imo'])
    
    lats = np.append(lats, None)
    lons = np.append(lons, None) 
    names = np.append(names, None) 
fig = px.line_geo(lat=lats, lon=lons, hover_name=names)
fig.show()

In [ ]:
dft = vessel.copy()
dft= dft.groupby('imo').agg({'lat':lambda x: list(x), 'lon':lambda x: list(x), 'geometry':lambda x: list(x), 'date':lambda x: list(x)})

def generate_linestring(geometry):
    #LineString([Point(0, 0), Point(1, 1)]).wkt
    try:
        # works also with comprehensive list
        return LineString(geometry).wkt
    except Exception as e:
        pass

dft['line'] = dft.apply(lambda x: generate_linestring(x['geometry']), axis=1)

dft.reset_index(inplace=True)
dft = dft.rename(columns = {'index':'imo'})

dft.tail()

In [ ]:
lats = []
lons = []
names = []
import numpy as np
import shapely.geometry
for index, row in dft.iterrows():
    try:
        lats = np.append(lats, row['lat'][0])

        lons = np.append(lons, row['lon'][0])

       # names = np.append(names, [row['imo']] * len(row['lat']))
        
        lats = np.append(lats, None)
        lons = np.append(lons, None)
       # names = np.append(names, None)
    except:
        pass

        
fig = px.line_geo(lat=lats, lon=lons)#, hover_name=names)
fig.show()

In [ ]:
import pandas as pd

us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")

us_cities = us_cities.query("State in ['New York', 'Ohio']")
print(us_cities.head())
import plotly.express as px

fig = px.line_mapbox(us_cities, lat="lat", lon="lon", color="State", zoom=3, height=300)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()


In [ ]:
def check_if_coord_in_poly(region, lon, lat):
    """
    Check if a coordinate (lat,long) is within a given polygon
    Used to check for an address in vienna geometry
    source: https://stackoverflow.com/questions/48097742/geopandas-point-in-polygon
    Return: True if Point within Polygon
    
    source: https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python
    
    """
    """
    _pnts = [Point(lon, lat)]
    poly = gpd.GeoSeries({'within': region})
    pnts = gpd.GeoDataFrame(geometry=_pnts, index=['Point to check'], crs=get_local_crs(lon, lat))
    pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in poly.items()})
    return pnts['within'].item()
    """
    point = Point(lon, lat)
    # print(region.contains(point))
    # return (point.within(region))
    #if region.contains(point):
    #    print(region.contains(point))
    return region.contains(point)
    
#for index, row in ports.iterrows():
#    vessel['close_to_harbor'] = vessel.apply(lambda x: check_if_coord_in_poly(row['bubble'], x['lon'], x['lat']), axis=1)
# vessel.head(3)

In [ ]:
# http://www.lngplants.com/conversiontables.html
Natural gas: 
HHV = 1027 Btu/ft3 = 38.3 MJ/m3
LHV    = 930 Btu/ft3 = 34.6 MJ/m3
 In the case of natural gas the ratio of HHV to LHV is approximately 1.108:1.
        

1,012      Btu/standard cubic foot methane             [Ref: Chemical Engineers’ Handbook. John H Perry, ed. McGraw-Hill Book Company: New York, 1963. Pg 9-9.]
19           1,050     Btu/standard cubic foot natural gas

Methane      0.941      Million Btu/Thousand Cubic Feet
http://www.nrel.gov/vehiclesandfuels/ngvtf/pdfs/lng_fuel_sys_tech_wegrzyn.pdf